In [ ]:
### TO DO: finish different types of analysis

In [1]:
### Given dictionary of changes, analyze the changes

In [1]:
import databaseOps, fields, utility
import numpy as np
import matplotlib.pyplot as plt
import json
from bs4 import BeautifulSoup, Tag
import copy
from datetime import datetime as dt
from sklearn.feature_extraction import DictVectorizer

In [2]:
import json
from pprint import pprint

#with open('changes.json') as data_file:    
with open('2016Q4_2017Q4_3MCOChanges.json') as data_file:
    jsonChanges = json.load(data_file)

In [3]:
numerical = ["transactionShares", "transactionPricePerShare", "sharesOwnedFollowingTransaction", "conversionOrExercisePrice", "underlyingSecurityShares"]

categorical = ["securityTitle", "transactionCode", "equitySwapInvolved", "transactionAcquiredDisposedCode", "directOrIndirectOwnership", "natureOfOwnership", "transactionTimelines", "underlyingSecurityTitlesecurityTitle"]

# categorical data w specified categories
typedCategorical = ["transactionCode", "equitySwapInvolved", "transactionAcquiredDisposedCode", "directOrIndirectOwnership", "transactionTimelines"]

date = ["transactionDate", "exerciseDate", "expirationDate"]

formPortion = ["head", "dt", "ndt"]

In [4]:
### Functions to perform basic analysis on dictionary of changes

def getCatAnalysis(changedFieldName, beforeVal, afterVal):
    catAnalysis = dict()
    catAnalysis["dataType"] = "categorical"
    catAnalysis["fieldName"] = changedFieldName
    
    if beforeVal == '':
        catAnalysis["changeType"] = "insert"
    elif afterVal == "":
        catAnalysis["changeType"] = "delete"
    else:
        catAnalysis["changeType"] = "update"
        
    if changedFieldName in typedCategorical:
        catAnalysis["updateVector"] = (beforeVal, afterVal)
    else:
        catAnalysis["editDistance"] = utility.editDistance(beforeVal, afterVal)
    
    return catAnalysis

def getNumAnalysis(changedFieldName, beforeVal, afterVal):
    numAnalysis = dict()
    numAnalysis["fieldName"] = changedFieldName
    numAnalysis["dataType"] = "numerical"
    
    if beforeVal == '':
        beforeVal = 0
        numAnalysis["changeType"] = "insert"
        afterVal = float(afterVal)
    elif afterVal == '':
        afterVal = 0
        numAnalysis["changeType"] = "delete"
        beforeVal = float(beforeVal)
        
    if beforeVal != "" and afterVal != "":
        beforeVal, afterVal = float(beforeVal), float(afterVal)
    
    change = afterVal - beforeVal
    if change != 0:
        numAnalysis["amountChanged"] = change        
        if afterVal > beforeVal:
            numAnalysis["changeDirection"] = "+" 
        else:
            numAnalysis["changeDirection"] = "-" 
        numAnalysis["percentChange"] = round(abs(change)/beforeVal*100, 3)      
        numAnalysis["changeType"] = "update"
    return numAnalysis

def getFootnoteAnalysis(analysisDic, beforeFoot, afterFoot, bFootContent, aFootContent):
    if len(beforeFoot) != 0 or len(afterFoot) != 0:
        if len(beforeFoot) > len(afterFoot):
            footChangeType = "footnote in 4"
        elif len(beforeFoot) < len(afterFoot):
            footChangeType = "footnote in 4/a"
        else:
            diffCount = sum([1 for i in range(len(beforeFoot)) if bFootContent[i] != aFootContent[i]])
            if diffCount != 0:
                footChangeType = "footnote content diff"
            else:
                if beforeFoot != afterFoot:
                    footChangeType = "footnote id diff"
                else:
                    footChangeType = ""
        if footChangeType != "":
            if "changeType" in analysisDic:
                analysisDic["changeType"] = [analysisDic["changeType"], footChangeType]
            else:
                analysisDic["changeType"] = footChangeType
    return analysisDic

def getDateAnalysis(changedFieldName, beforeVal, afterVal):
    dateAnalysis = dict()
    dateAnalysis["dataType"] = "date"
    if beforeVal == "":
        dateAnalysis["changeType"] = "insert"
    if afterVal == "":
        dateAnalysis["changeType"] = "delete"
    if beforeVal != "" and afterVal != "":
        beforeDate = dt.strptime(beforeVal, "%Y-%m-%d")
        afterDate = dt.strptime(afterVal, "%Y-%m-%d")
        dateDiff = beforeDate - afterDate
        if dateDiff != 0:
            dateAnalysis["changeType"] = "update"
            if beforeDate > afterDate:
                dateAnalysis["changeDirection"] = "+" #changed date to later
            elif beforeDate < afterDate:
                dateAnalysis["changeDirection"] = "-"
            dateAnalysis["amountChanged"] = dateDiff.total_seconds()/60/60/24 #days
    return dateAnalysis


def runAnalysis(fileDic):
    files = copy.deepcopy(fileDic)
    for file in files:
        changeDic = files[file]
        for portion in formPortion:
            for rowKey in list(changeDic[portion].keys()):
                row = changeDic[portion][rowKey]
                
                for changedFieldName in list(row.keys()):
                    footnoteChangeOnly = False

                    if "value" in row[changedFieldName]['4']:
                        beforeVal = row[changedFieldName]['4']['value']
                    else:
                        beforeVal = ""
                    if "value" in row[changedFieldName]['4A']:
                        afterVal = row[changedFieldName]['4A']['value'] 
                    else: 
                        afterVal = ""

                    beforeFoot, afterFoot = None, None
                    if isinstance(beforeVal, str) and ("<value>" in beforeVal or "footnoteId" in beforeVal):
                        beforeSoup = BeautifulSoup(beforeVal)
                        beforeVal = ''.join([str(val.text) for val in beforeSoup.findAll('value')])
                        beforeFoot = [footnote.get('id') for footnote in beforeSoup.findAll('footnoteid')] #soup made it lowercased
                        if beforeVal == '':
                            footnoteChangeOnly = True

                    if isinstance(afterVal, str) and ("<value>" in afterVal or "footnoteId" in afterVal):
                        afterSoup = BeautifulSoup(afterVal)
                        afterVal = ''.join([str(val.text) for val in afterSoup.findAll('value')])
                        afterFoot = [footnote.get('id') for footnote in afterSoup.findAll('footnoteid')]

                        if footnoteChangeOnly:
                            if afterVal != '':
                                footnoteChangeOnly = False

                    bFootContent = [row[changedFieldName]['4'][fid] for fid in beforeFoot]
                    aFootContent = [row[changedFieldName]['4A'][fid] for fid in afterFoot]
                        
                    if footnoteChangeOnly:
                        tempDic = dict()
                        tempDic["dataType"] = "footnote"
                        row[changedFieldName]["analysis"] = getFootnoteAnalysis(tempDic, beforeFoot, afterFoot, bFootContent, aFootContent)
                    else: 
                        if changedFieldName in categorical:
                            analysisDic = getCatAnalysis(changedFieldName, beforeVal, afterVal)
                            row[changedFieldName]["analysis"] = getFootnoteAnalysis(analysisDic, beforeFoot, afterFoot, bFootContent, aFootContent)
                        elif changedFieldName in numerical:
                            #if "footnoteId" not in beforeVal and "footnoteId" not in afterVal:
                            analysisDic = getNumAnalysis(changedFieldName, beforeVal, afterVal)
                            row[changedFieldName]["analysis"] = getFootnoteAnalysis(analysisDic, beforeFoot, afterFoot, bFootContent, aFootContent)
                        elif changedFieldName in date:
                            analysisDic = getDateAnalysis(changedFieldName, beforeVal, afterVal)
                            row[changedFieldName]["analysis"] = getFootnoteAnalysis(analysisDic, beforeFoot, afterFoot, bFootContent, aFootContent)


#     print(json.dumps(files, indent=2))
    return files

In [5]:
analysis = runAnalysis(jsonChanges)
analysis

{'66740/0001127602-17-004847': {'accNum': '66740/0001127602-17-004744',
  'dt': {'1-transaction': {'securityTitle': {'4': {'F3': 'This option becomes exercisable in equal installments on each of the first three anniversaries of the grant date (2/7/2017).',
      'value': '<value>Non-qualified Stock Option (Right to Buy)</value><footnoteId id="F3"/>'},
     '4A': {'value': '<value>Non-qualified Stock Option (Right to Buy)</value>'},
     'analysis': {'changeType': ['update', 'footnote in 4'],
      'dataType': 'categorical',
      'editDistance': 0,
      'fieldName': 'securityTitle'}}}},
  'head': {},
  'ndt': {'1-transaction': {'sharesOwnedFollowingTransaction': {'4': {'value': '<value>10860</value>'},
     '4A': {'F1': "Due to a computational error caused by a third-party vendor's filing software, the reporting person is refiling a report filed earlier today, solely to correct total holdings reported in Column 5 of Table 1.",
      'value': '<value>16812</value><footnoteId id="F1"/>'

In [68]:
### Functions to summarize changes & analyses 

dataType = ["categorical", "numerical", "footnote", "date"]
categoricalAnalysisType = {
    "changeDirection": ["+", "-"],
    "changeType": ["update", "insert", "delete", "footnote in 4", "footnote in 4/a", "footnote content diff", "footnote id diff"]
}

numericalAnalysisType = ["amountChanged", "percentChange"]

changeTypeVect = {
    "update": 1,
    "insert": 2,
    "delete": 3,
    "footnote in 4": 30,
    "footnote in 4/a": 40,
    "footnote content diff": 50,
    "footnote id diff": 60,
}
dataTypeVect = {
    "numerical": 0,
    "categorical": 1,
    "date": 2,
    "footnote" : 3
}

"""
Get all analyses dictionaries
- outputs: analyses, accNums, rowIds
    analyses['portion']['data'] = raw data of analyses
    analyses['portion']['metadata'] = ex: counts for categorical analyses
    accNums['portion']: list of accNums of returned analyses
    rowIds['portion']: list of rowId of returned analyses (used to index into database)
- params: 
    dataType: "categorical" | "numerical"
    analysisType: see fields of analysisDictionary (ex:"amountChanged")
    analysisSubType: ex for analysisType=changeType, analysisSubType=1
    toVect: change output['data'] to numerical values only for dictvectorizer
"""
def getAllAnalyses(files, dataType=False, analysisType=False, analysisSubType=False, toVect=False):
    analyses = {}
    analyses["dt"], analyses["ndt"] = {}, {}
    analyses["dt"]["data"], analyses["ndt"]["data"] = [], []
    if analysisType in categoricalAnalysisType:    
        analyses["dt"]["metadata"], analyses["ndt"]["metadata"] = {}, {}
        for option in categoricalAnalysisType[analysisType]:
            analyses["dt"]["metadata"][option], analyses["ndt"]["metadata"][option] = 0, 0
    
    accNums, rowIds = {}, {}
    accNums["dt"], accNums["ndt"] = [], []
    rowIds["dt"], rowIds["ndt"] = [], []
    
    files = copy.deepcopy(files)
    for file in files:
        changeDic = files[file]
        for portion in formPortion:
            for rowKey in list(changeDic[portion].keys()):
                row = changeDic[portion][rowKey]
                for changedFieldName in list(row.keys()):
                    if "analysis" in row[changedFieldName] and (not dataType or dataType == row[changedFieldName]["analysis"]["dataType"]):
                        if analysisType and (analysisType in row[changedFieldName]["analysis"]):
                            data = row[changedFieldName]["analysis"][analysisType]
                            analyses[portion]["data"].append(data)
                            if analysisType in categoricalAnalysisType:
                                if analysisType == "changeType":
                                    for changeType in data:
                                        analyses[portion]["metadata"][changeType] += 1
                                else:
                                    analyses[portion]["metadata"][data] += 1
                            
                        elif not analysisType:
                            data = row[changedFieldName]["analysis"]
                            analyses[portion]["data"].append(data)
                        accNums[portion].append(file)
                        if "-transaction" in rowKey:
                            rowIdx = rowKey.replace("-transaction", "")
                            thisRowId = file+"-"+rowIdx
                        else:
                            rowIdx = rowKey.replace("-holding", "")
                            thisRowId = file+"-"+rowIdx+"-h"
                        rowIds[portion].append(thisRowId)
#     if analysisType in numericalAnalysisType:
#         analyses["metadata"] = getNumMetadata(analyses["data"])
    
    if toVect:
        for portion in list(analyses.keys()):
            for dic in analyses[portion]['data']:
                if type(dic["changeType"]) == list:
                    dic["changeType"] = sum([changeTypeVect[val] for val in dic["changeType"]])
                else:
                    dic["changeType"] = changeTypeVect[dic["changeType"]]
                if dic["dataType"] == "numerical":
                    dic["fieldName"] = numerical.index(dic["fieldName"])
                elif dic["dataType"] == "categorical":
                    dic["fieldName"] = categorical.index(dic["fieldName"])
                else:
                    dic["fieldName"] = date.index(dic["fieldName"])
                dic["dataType"] = dataTypeVect[dic["dataType"]]
                if "changeDirection" in dic:
                    dic["changeDirection"] = 1 if dic["changeDirection"] == "+" else 2
    return analyses, accNums, rowIds

In [75]:
# get all Analyses
getAllAnalyses(analysis)

({'dt': {'data': [{'changeType': ['update', 'footnote in 4'],
     'dataType': 'categorical',
     'editDistance': 0,
     'fieldName': 'securityTitle'},
    {'changeType': ['update', 'footnote in 4'],
     'dataType': 'categorical',
     'editDistance': 0,
     'fieldName': 'securityTitle'},
    {'changeType': ['update', 'footnote in 4'],
     'dataType': 'categorical',
     'editDistance': 0,
     'fieldName': 'securityTitle'},
    {'changeType': ['update', 'footnote in 4'],
     'dataType': 'categorical',
     'editDistance': 0,
     'fieldName': 'securityTitle'},
    {'changeType': ['update', 'footnote in 4'],
     'dataType': 'categorical',
     'editDistance': 0,
     'fieldName': 'securityTitle'},
    {'changeType': ['update', 'footnote in 4'],
     'dataType': 'categorical',
     'editDistance': 0,
     'fieldName': 'securityTitle'},
    {'changeType': ['update', 'footnote in 4'],
     'dataType': 'categorical',
     'editDistance': 0,
     'fieldName': 'securityTitle'},
    {'

In [76]:
# Get numerical analyses only
getAllAnalyses(analysis, "numerical")[0]

{'dt': {'data': []},
 'ndt': {'data': [{'amountChanged': 5737.849900000001,
    'changeDirection': '+',
    'changeType': ['update', 'footnote in 4'],
    'dataType': 'numerical',
    'fieldName': 'sharesOwnedFollowingTransaction',
    'percentChange': 52.835},
   {'amountChanged': 39.51400000000001,
    'changeDirection': '+',
    'changeType': ['update', 'footnote content diff'],
    'dataType': 'numerical',
    'fieldName': 'transactionShares',
    'percentChange': 22.646},
   {'amountChanged': 5951.849900000001,
    'changeDirection': '+',
    'changeType': 'update',
    'dataType': 'numerical',
    'fieldName': 'sharesOwnedFollowingTransaction',
    'percentChange': 54.805},
   {'changeType': 'footnote in 4',
    'dataType': 'numerical',
    'fieldName': 'transactionShares'},
   {'amountChanged': 838.7467000000001,
    'changeDirection': '+',
    'changeType': ['update', 'footnote in 4'],
    'dataType': 'numerical',
    'fieldName': 'sharesOwnedFollowingTransaction',
    'percent

In [78]:
# Get numerical analyses with "changeDirection"
print(getAllAnalyses(analysis, "numerical", "changeDirection")[0])

# Get catergorical analyses with "changeType"
print(getAllAnalyses(analysis, "categorical", "changeType")[0]["ndt"]["metadata"])

{'ndt': {'data': ['+', '+', '+', '+', '-', '+', '+', '-', '+', '+', '+', '+', '+', '+', '+', '+', '+', '-', '+', '+', '+', '+', '+', '-', '+', '+', '-', '+', '+', '+', '+', '+', '-', '+', '-', '+', '-', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '-', '+', '+', '+', '+', '+', '+', '-', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '-', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+', '+'], 'metadata': {'+': 67, '-': 11}}, 'dt': {'data': [], 'metadata': {'+': 0, '-': 0}}}
{'footnote in 4': 0, 'footnote in 4/a': 0, 'insert': 0, 'footnote id diff': 0, 'delete': 0, 'update': 0, 'footnote content diff': 0}


In [85]:
"""
Get labels to use for training
- input: changeDictionary
- output: numLabels, catLabels 
    numLabels/catLabels[dt/ndt][labels] = label vectors
    numLabels/catLabels[dt/ndt][featureNames] = features names
    numLabels/catLabels[dt/ndt][accNums] = accNum of each data
    numLabels/catLabels[dt/ndt][rowIds] = id of each data (for database)
"""
def getLabels(jsonChanges):
    analysis = runAnalysis(jsonChanges)

    numAnalyses, numAccNums, numRowIds = getAllAnalyses(analysis, "numerical", toVect=True)
    catAnalyses, catAccNums, catRowIds = getAllAnalyses(analysis, "categorical", toVect=True)

    numDTRecords, numNDTRecords = numAnalyses["dt"]["data"], numAnalyses["ndt"]["data"]
    catDTRecords, catNDTRecords = catAnalyses["dt"]["data"], catAnalyses["ndt"]["data"]

    vect = DictVectorizer(sparse=False)
    
    numLabels = {}
    numLabels["dt"], numLabels["ndt"] = {}, {}
    numLabels["dt"]["labels"], numLabels["dt"]["featureNames"] =  np.array([]), np.array([])
    numLabels["ndt"]["labels"], numLabels["ndt"]["featureNames"] =  np.array([]), np.array([])
    catLabels = copy.deepcopy(numLabels)
    
    numLabels["dt"]["accNums"], numLabels["ndt"]["accNums"] = numAccNums['dt'], numAccNums['ndt']
    catLabels["dt"]["accNums"], catLabels["ndt"]["accNums"] = catAccNums['dt'], catAccNums['ndt']
    
    numLabels["dt"]["rowIds"], numLabels["ndt"]["rowIds"] = numRowIds['dt'], numRowIds['ndt']
    catLabels["dt"]["rowIds"], catLabels["ndt"]["rowIds"] = catRowIds['dt'], catRowIds['ndt']
    
    if len(numDTRecords) > 0:
        numLabels["dt"]["labels"] = vect.fit_transform(numDTRecords)
        numLabels["dt"]["featureNames"] = vect.get_feature_names()
    if len(numNDTRecords) > 0:
        numLabels["ndt"]["labels"] = vect.fit_transform(numNDTRecords)
        numLabels["ndt"]["featureNames"] = vect.get_feature_names()
    if len(catDTRecords) > 0:
        catLabels["dt"]["labels"] = vect.fit_transform(catDTRecords)
        catLabels["dt"]["featureNames"] = vect.get_feature_names()
    if len(catNDTRecords) > 0:
        catLabels["dt"]["labels"] = vect.fit_transform(catNDTRecords)
        catLabels["dt"]["featureNames"] = vect.get_feature_names()

    return numLabels, catLabels

In [88]:
numLabels, catLabels = getLabels(jsonChanges)
numLabels["ndt"]

{'accNums': ['66740/0001127602-17-007785',
  '66740/0001127602-17-007785',
  '66740/0001127602-17-007785',
  '66740/0001127602-17-007785',
  '66740/0001127602-17-007816',
  '66740/0001127602-17-007816',
  '66740/0001127602-17-007816',
  '66740/0001127602-17-007816',
  '66740/0001127602-17-007803',
  '66740/0001127602-17-007803',
  '66740/0001127602-17-007803',
  '66740/0001127602-17-007803',
  '66740/0001127602-17-007803',
  '66740/0001127602-17-007787',
  '66740/0001127602-17-007787',
  '66740/0001127602-17-007787',
  '66740/0001127602-17-007787',
  '66740/0001127602-17-004878',
  '66740/0001127602-17-004878',
  '66740/0001127602-17-004878',
  '66740/0001127602-17-004878',
  '66740/0001127602-17-004878',
  '66740/0001127602-17-004870',
  '66740/0001127602-17-004870',
  '66740/0001127602-17-004870',
  '66740/0001127602-17-004870',
  '66740/0001127602-17-004870',
  '66740/0001127602-17-007827',
  '66740/0001127602-17-007827',
  '66740/0001127602-17-007827',
  '66740/0001127602-17-007827

In [35]:

def getNumChanges(files, dataType, analysisType=False, analysisSubType=False):
    analyses = getAllAnalyses(files, dataType, analysisType)
    if analysisType is False:
        print("Number of {} changes: {}".format(dataType, len(analyses["data"])))
    if analysisType is not False:
        print("Number of {} changes in {}: {}".format(dataType, analysisType, len(analyses["data"])))
        if analysisType in categoricalAnalysisType:
            options = categoricalAnalysisType[analysisType]
            for option in options:
                print("Number of {} changes with {} of {}: {}".format(dataType, analysisType, option, analyses["metadata"][option]))
            
# az = [getNumChanges(analysis, i) for i in dataType]
# getNumChanges(analysis, "numerical", "amountChanged")
# getNumChanges(analysis, "numerical", "changeDirection")

TypeError: tuple indices must be integers or slices, not str

In [33]:
# def getCatStats(dic):

def getNumStats(data, dataType, analysisType): #for distributions
    print("Mean", np.mean(data))
    print("Median", np.median(data))
    print("25% Percentile", np.percentile(data, 25))
    print("75% Percentile", np.percentile(data, 75))
    print("Standard deviation", np.std(data))
    hist, bins = np.histogram(data)
    plt.hist(data, bins)
    plt.title("Distribution of {} changes in {}".format(dataType, analysisType))
    plt.show()

def getNumMetadata(files, dataType, analysisType=False, analysisSubType=False):
    data = getAllAnalyses(files, dataType, analysisType)["data"]
    getStats(data)
    
# getNumMetadata(analysis, "numerical", "amountChanged")

In [73]:
realChanges = {
  "1545654/000122520817001902/0001225208-17-001902": {
    "dt": {},
    "accNum": "1545654/0001225208-17-001730",
    "head": {},
    "ndt": {
      "1-transaction": {
        "transactionShares": {
          "4A": {
            "value": "<value>615.0000</value><footnoteId id=\"F2\"/>",
            "F2": "The number of shares reflected in the Reporting Person's Form 4 (filed on January 26, 2017) was incorrectly reported. The correct number of shares is reflected in this amendment."
          },
          "4": {
            "value": "<value>678.0000</value>"
          }
        },
        "sharesOwnedFollowingTransaction": {
          "4A": {
            "value": "<value>79801.6695</value>"
          },
          "4": {
            "value": "<value>79738.6695</value>"
          }
        }
      }
    }
  },
  "1253352/000114036117015044/0001140361-17-015044": {
    "dt": {
      "2-transaction": {
        "expirationDate": {
          "4A": {
            "F4": "This option, representing a right to purchase a total of 26,250 shares, was granted on 1/1/2014 and vests over 4 years in equal annual installments commencing on the first anniversary of the grant date.",
            "value": "<footnoteId id=\"F4\"/>"
          },
          "4": {
            "value": "<footnoteId id=\"F3\"/>",
            "F3": "This option, representing a right to purchase a total of 26,250 shares, was granted on 1/1/2014 and vests over 4 years in equal annual installments commencing on the first anniversary of the grant date."
          }
        },
        "exerciseDate": {
          "4A": {
            "F4": "This option, representing a right to purchase a total of 26,250 shares, was granted on 1/1/2014 and vests over 4 years in equal annual installments commencing on the first anniversary of the grant date.",
            "value": "<footnoteId id=\"F4\"/>"
          },
          "4": {
            "value": "<footnoteId id=\"F3\"/>",
            "F3": "This option, representing a right to purchase a total of 26,250 shares, was granted on 1/1/2014 and vests over 4 years in equal annual installments commencing on the first anniversary of the grant date."
          }
        }
      },
      "1-transaction": {
        "expirationDate": {
          "4A": {
            "value": "<footnoteId id=\"F3\"/>",
            "F3": "On January 4, 2016, the reporting person received restricted stock units under Iradimed Corporation's 2014 Equity Incentive Plan. The restricted stock units vest in two equal annual installments beginning on January 4, 2017. Upon vesting, the reporting person will receive a number of shares of common stock equal to the number of restricted stock units that have vested."
          },
          "4": {
            "value": "<footnoteId id=\"F2\"/>",
            "F2": "On January 4, 2016, the reporting person received restricted stock units under Iradimed Corporation's 2014 Equity Incentive Plan. The restricted stock units vest in two equal annual installments beginning on January 4, 2017. Upon vesting, the reporting person will receive a number of shares of common stock equal to the number of restricted stock units that have vested."
          }
        },
        "conversionOrExercisePrice": {
          "4A": {
            "value": "<footnoteId id=\"F2\"/>",
            "F2": "Restricted stock units convert into common stock on a one-for-one basis."
          },
          "4": {
            "F1": "Restricted stock units convert into common stock on a one-for-one basis.",
            "value": "<footnoteId id=\"F1\"/>"
          }
        },
        "exerciseDate": {
          "4A": {
            "value": "<footnoteId id=\"F3\"/>",
            "F3": "On January 4, 2016, the reporting person received restricted stock units under Iradimed Corporation's 2014 Equity Incentive Plan. The restricted stock units vest in two equal annual installments beginning on January 4, 2017. Upon vesting, the reporting person will receive a number of shares of common stock equal to the number of restricted stock units that have vested."
          },
          "4": {
            "value": "<footnoteId id=\"F2\"/>",
            "F2": "On January 4, 2016, the reporting person received restricted stock units under Iradimed Corporation's 2014 Equity Incentive Plan. The restricted stock units vest in two equal annual installments beginning on January 4, 2017. Upon vesting, the reporting person will receive a number of shares of common stock equal to the number of restricted stock units that have vested."
          }
        }
      }
    },
    "accNum": "1253352/0001140361-17-001007",
    "head": {},
    "ndt": {
      "2-transaction": {
        "sharesOwnedFollowingTransaction": {
          "4A": {
            "value": "<value>10960</value>"
          },
          "4": {
            "value": "<value>10340</value>"
          }
        }
      },
      "1-transaction": {
        "transactionShares": {
          "4A": {
            "F1": "This amendment is being filed to remove reference to improperly withheld shares of common stock issued to the reporting person pursuant to the vesting of restricted stock units. Due to an administrative error, upon the vesting of restricted stock units on 1/4/2017, 620 shares of common stock were withheld to satisfy tax withholding obligations as if the reporting person were an employee of the issuer. This error was corrected and the improperly-withheld 620 shares of common stock were issued to the reporting person on 4/3/2017.",
            "value": "<value>1898</value><footnoteId id=\"F1\"/>"
          },
          "4": {
            "value": "<value>1898</value>"
          }
        },
        "transactionPricePerShare": {
          "4A": {
            "value": "<footnoteId id=\"F2\"/>",
            "F2": "Restricted stock units convert into common stock on a one-for-one basis."
          },
          "4": {
            "F1": "Restricted stock units convert into common stock on a one-for-one basis.",
            "value": "<footnoteId id=\"F1\"/>"
          }
        }
      }
    }
  },
  "1444236/000120919117016922/0001209191-17-016922": {
    "dt": {
      "1-transaction": {
        "exerciseDate": {
          "4A": {
            "value": "<value>2018-02-23</value><footnoteId id=\"F3\"/>",
            "F3": "This option vests in four equal annual installments beginning February 23, 2018."
          },
          "4": {
            "value": "<value>2018-02-23</value><footnoteId id=\"F2\"/>",
            "F2": "This option vests in four equal annual installments beginning February 23, 2018."
          }
        }
      }
    },
    "accNum": "1444236/0001209191-17-014948",
    "head": {},
    "ndt": {
      "1-transaction": {
        "sharesOwnedFollowingTransaction": {
          "4A": {
            "value": "<value>17625</value><footnoteId id=\"F2\"/>",
            "F2": "This Amendment is being filed to correct an error in the number of shares of Series A Common Stock beneficially owned after the reported transaction."
          },
          "4": {
            "value": "<value>45833</value>"
          }
        }
      }
    }
  }
}

sampleChanges = {
    "num": {
    "dt": {
      "1-transaction": {
        "securityTitle": {
          "4A": {
            "value": "some title"
          },
          "4": {
            "value": "new title"
          }
        }, "directOrIndirectOwnership": {
          "4A": {
            "value": "<value>D</value>"
          },
          "4": {
            "value": "<value>I</value>"
          }
        },
        "conversionOrExercisePrice": {
          "4A": {
            "value": "<footnoteId id=\"F2\"/>",
            "F2": "Restricted stock units convert into common stock on a one-for-one basis."
          },
          "4": {
            "F1": "Restricted stock units convert into common stock on a one-for-one basis.",
            "value": "<footnoteId id=\"F1\"/>"
          }
        }
      }
    },
    "accNum": "1444236/0001209191-17-014948",
    "head": {},
    "ndt": {
      "1-transaction": {
        "sharesOwnedFollowingTransaction": {
          "4A": {
            "value": "<value>17625</value><footnoteId id=\"F2\"/>",
            "F2": "This Amendment is being filed to correct an error in the number of shares of Series A Common Stock beneficially owned after the reported transaction."
          },
          "4": {
            "value": "<value>45833</value>"
          }
        }, "conversionOrExercisePrice": {
          "4A": {
            "value": "32323"
          },
          "4": {
            "value": "2"
          }
        }
      }
    }
}    
}

In [3]:
import sys
!{sys.executable} -m pip install pandas

    100% |████████████████████████████████| 14.4MB 788kB/s  a 0:00:011
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
